<a href="https://colab.research.google.com/github/KoneruGouthami/Digital-Resume/blob/master/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INSTALLATION**

In [ ]:
!pip install ultralytics

In [ ]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

**MOUNTING THE COLAB TO DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**TRAINING THE DATASET**

In [ ]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

**Splitting  dataset into training and testing sets, optionally including negative images in the training set, and preparing the data for training and evaluating the machine learning model.**

In [ ]:
import os
import random
import shutil
from tqdm import tqdm

def train_test_split(path, neg_path=None, split=0.2):
    print("------ PROCESS STARTED -------")

    if not path.endswith('/'):
        path += '/'

    files = list(set([name[:-4] for name in os.listdir(path) if name.endswith('.jpg')]))  ## Only consider .jpg files

    print(f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories
    os.makedirs(train_path_img, exist_ok=True)
    os.makedirs(train_path_label, exist_ok=True)
    os.makedirs(val_path_img, exist_ok=True)
    os.makedirs(val_path_label, exist_ok=True)

    for filex in tqdm(files[:train_size]):
        if filex == 'classes':
            continue
        shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
        # Check if corresponding .txt file exists before copying
        if os.path.exists(path + filex + '.txt'):
            shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')

    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path) if name.endswith('.jpg')])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
        if filex == 'classes':
            continue
        # print("running")
        shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
        # Check if corresponding .txt file exists before copying
        if os.path.exists(path + filex + '.txt'):
            shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## splitting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/yolov8/data') ### without negative images

------ PROCESS STARTED -------
--- This folder has a total number of 90 images---


100%|██████████| 72/72 [00:03<00:00, 22.66it/s]


------ Training data created with 80% split 72 images -------


100%|██████████| 18/18 [00:00<00:00, 111.11it/s]

------ Testing data created with a total of 18 images ----------
------ TASK COMPLETED -------


In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.48 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.3/107.7 GB disk)


In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/yolov8/dataset.yaml epochs=10 imgsz=640 plots=True batch=16 project=/content/drive/MyDrive/yolov8/training_results name=TeleICU

100% 21.5M/21.5M [00:00<00:00, 237MB/s]
Ultralytics YOLOv8.2.48 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/yolov8/dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8/training_results name=TeleICU, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False,

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results name=TeleICU/train3/weights/best.pt conf=0.35 source=/content/drive/MyDrive/yolov8/test_images

Ultralytics YOLOv8.2.48 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 11127519 parameters, 0 gradients, 28.4 GFLOPs

image 1/15 /content/drive/MyDrive/yolov8/test_images/1051518295-preview_000.jpg: 384x640 1 Patient feeling discomfort, 399.0ms
image 2/15 /content/drive/MyDrive/yolov8/test_images/1051518295-preview_180.jpg: 384x640 1 Patient feeling discomfort, 365.6ms
image 3/15 /content/drive/MyDrive/yolov8/test_images/1065887893-preview_016.jpg: 384x640 1 Patient feeling discomfort, 375.4ms
image 4/15 /content/drive/MyDrive/yolov8/test_images/1065887893-preview_105.jpg: 384x640 1 Patient feeling discomfort, 355.8ms
image 5/15 /content/drive/MyDrive/yolov8/test_images/1066004173-preview_052.jpg: 384x640 (no detections), 382.0ms
image 6/15 /content/drive/MyDrive/yolov8/test_images/1066004173-preview_083.jpg: 384x640 1 Patient feeling discomfort, 389.3ms
image 7/15 /content/drive/MyDrive/yolov8/test_images/1072459619-preview_000.jpg: 384x

In [ ]:
!cp -r /content/runs/detect/predict2 /content/drive/MyDrive/yolov8/output

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results name=TeleICU/train3/weights/best.pt conf=0.77 source=/content/drive/MyDrive/yolov8/videos

Ultralytics YOLOv8.2.48 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 11127519 parameters, 0 gradients, 28.4 GFLOPs

video 1/8 (frame 1/467) /content/drive/MyDrive/yolov8/videos/1.mp4: 384x640 1 Patient feeling discomfort, 418.9ms
video 1/8 (frame 2/467) /content/drive/MyDrive/yolov8/videos/1.mp4: 384x640 1 Patient feeling discomfort, 390.8ms
video 1/8 (frame 3/467) /content/drive/MyDrive/yolov8/videos/1.mp4: 384x640 1 Patient feeling discomfort, 367.4ms
video 1/8 (frame 4/467) /content/drive/MyDrive/yolov8/videos/1.mp4: 384x640 1 Patient feeling discomfort, 380.9ms
video 1/8 (frame 5/467) /content/drive/MyDrive/yolov8/videos/1.mp4: 384x640 1 Patient feeling discomfort, 384.3ms
video 1/8 (frame 6/467) /content/drive/MyDrive/yolov8/videos/1.mp4: 384x640 1 Patient feeling discomfort, 393.5ms
video 1/8 (frame 7/467) /content/drive/MyDrive/yolov8/videos/1.mp4: 384x640 1 Patient feeling discomfort, 607.9ms
video 1/8 (frame 8/467) /content/dri

In [ ]:
!cp -r /content/runs/detect/predict2 /content/drive/MyDrive/yolov8/output

# ***TESTING***

In [ ]:
import os
from ultralytics import YOLO
import cv2

# Load the custom trained YOLOv8 model
model = YOLO('/content/drive/MyDrive/yolov8/training_results name=TeleICU/train3/weights/best.pt')

# Video source and output directory
video_path = "/content/drive/MyDrive/yolov8/videos/9-preview.mp4"
output_dir = "/content/drive/MyDrive/yolov8/output"
os.makedirs(output_dir, exist_ok=True)

# Perform tracking with the model
results = model.track(video_path, stream=True, show=False)  # Use stream=True to handle large files

# Process the results and save the annotated frames
for i, result in enumerate(results):
    frame = result.orig_img  # Original image
    annotated_frame = result.plot()  # Plot the detections

    # Save the annotated frame
    frame_path = os.path.join(output_dir, f"frame_{i+1:04d}.jpg")
    cv2.imwrite(frame_path, annotated_frame)

print("Processing complete. Annotated frames saved in:", output_dir)



video 1/1 (frame 1/340) /content/drive/MyDrive/yolov8/videos/9-preview.mp4: 384x640 1 Patient feeling discomfort, 717.7ms
video 1/1 (frame 2/340) /content/drive/MyDrive/yolov8/videos/9-preview.mp4: 384x640 1 Patient feeling discomfort, 614.4ms
video 1/1 (frame 3/340) /content/drive/MyDrive/yolov8/videos/9-preview.mp4: 384x640 1 Patient feeling discomfort, 656.5ms
video 1/1 (frame 4/340) /content/drive/MyDrive/yolov8/videos/9-preview.mp4: 384x640 (no detections), 597.8ms
video 1/1 (frame 5/340) /content/drive/MyDrive/yolov8/videos/9-preview.mp4: 384x640 1 Patient feeling discomfort, 393.4ms
video 1/1 (frame 6/340) /content/drive/MyDrive/yolov8/videos/9-preview.mp4: 384x640 1 Patient feeling discomfort, 381.1ms
video 1/1 (frame 7/340) /content/drive/MyDrive/yolov8/videos/9-preview.mp4: 384x640 1 Patient feeling discomfort, 401.0ms
video 1/1 (frame 8/340) /content/drive/MyDrive/yolov8/videos/9-preview.mp4: 384x640 1 Patient feeling discomfort, 380.2ms
video 1/1 (frame 9/340) /content/dri

In [ ]:
import os
from ultralytics import YOLO
import cv2

# Load the custom trained YOLOv8 model
model = YOLO('/content/drive/MyDrive/yolov8/training_results name=TeleICU/train3/weights/best.pt')

# Video source and output path
video_path = "/content/drive/MyDrive/yolov8/videos/9-preview.mp4"
output_video_path = "/content/drive/MyDrive/yolov8/output_video/annotated_video.mp4"

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(output_video_path), exist_ok=True)

# Open the video file
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 file
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Perform tracking with the model and save the annotated frames
results = model.track(video_path, stream=True, show=False)  # Use stream=True to handle large files

for result in results:
    annotated_frame = result.plot()  # Plot the detections
    out.write(annotated_frame)  # Write the frame to the output video

# Release the video objects
cap.release()
out.release()

print("Processing complete. Annotated video saved as:", output_video_path)



video 1/1 (frame 1/340) /content/drive/MyDrive/yolov8/videos/9-preview.mp4: 384x640 1 Patient feeling discomfort, 423.1ms
video 1/1 (frame 2/340) /content/drive/MyDrive/yolov8/videos/9-preview.mp4: 384x640 1 Patient feeling discomfort, 372.6ms
video 1/1 (frame 3/340) /content/drive/MyDrive/yolov8/videos/9-preview.mp4: 384x640 1 Patient feeling discomfort, 386.8ms
video 1/1 (frame 4/340) /content/drive/MyDrive/yolov8/videos/9-preview.mp4: 384x640 (no detections), 379.9ms
video 1/1 (frame 5/340) /content/drive/MyDrive/yolov8/videos/9-preview.mp4: 384x640 1 Patient feeling discomfort, 375.9ms
video 1/1 (frame 6/340) /content/drive/MyDrive/yolov8/videos/9-preview.mp4: 384x640 1 Patient feeling discomfort, 378.9ms
video 1/1 (frame 7/340) /content/drive/MyDrive/yolov8/videos/9-preview.mp4: 384x640 1 Patient feeling discomfort, 380.5ms
video 1/1 (frame 8/340) /content/drive/MyDrive/yolov8/videos/9-preview.mp4: 384x640 1 Patient feeling discomfort, 384.4ms
video 1/1 (frame 9/340) /content/dri